In [2]:
from pychorus.similarity_matrix import TimeTimeSimilarityMatrix, TimeLagSimilarityMatrix, Line
from pychorus.constants import N_FFT, SMOOTHING_SIZE_SEC, LINE_THRESHOLD, MIN_LINES, \
    NUM_ITERATIONS, OVERLAP_PERCENT_MARGIN
from pychorus.helpers import *
import os
from IPython.display import Audio


In [3]:
def display_audio(audio, sr = None):
    return Audio(data = audio, rate = sr)

In [4]:
path = os.path.join('audio', 'audio1.wav')
clip_length = 10

chroma, song_wav_data, sr, song_length_sec = create_chroma(path)
num_samples = chroma.shape[1]

time_time_similarity = TimeTimeSimilarityMatrix(chroma, sr)
time_lag_similarity = TimeLagSimilarityMatrix(chroma, sr)

# Denoise the time lag matrix
chroma_sr = num_samples / song_length_sec
smoothing_size_samples = int(SMOOTHING_SIZE_SEC * chroma_sr)
time_lag_similarity.denoise(time_time_similarity.matrix,
                            smoothing_size_samples)

# Detect lines in the image
clip_length_samples = clip_length * chroma_sr
candidate_rows = local_maxima_rows(time_lag_similarity.matrix)
lines = detect_lines(time_lag_similarity.matrix, candidate_rows,
                        clip_length_samples)
if len(lines) == 0:
    print("No choruses were detected. Try a smaller search duration")
line_scores = count_overlapping_lines(
    lines, OVERLAP_PERCENT_MARGIN * clip_length_samples,
    clip_length_samples)

lines_to_sort = []
for line in line_scores:
    lines_to_sort.append((line, line_scores[line], line.end - line.start))

lines_to_sort.sort(key=lambda x: (x[1], x[2]), reverse=True)

best_chorus = best_segment(line_scores)
    
r = best_chorus.start / chroma_sr

In [11]:
frames_per_sample = int(len(song_wav_data)/num_samples+1)
lines_to_sort
#num_samples

[(Line (699 787 458), 1, 88),
 (Line (700 787 141), 1, 87),
 (Line (373 600 316), 0, 227)]

In [13]:
line, rep, length = lines_to_sort[0]
display_audio(song_wav_data[line.start*frames_per_sample : line.end*frames_per_sample], sr)

In [15]:
print("Best chorus found at {0:g} min {1:.2f} sec".format(
        r // 60, r % 60))

Best chorus found at 2 min 9.77 sec
